In [ ]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.9/671.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
from atproto import Client

client = Client()

# Use the correct parameters for login: handle, password
# session_string is not meant for passing the service URL
client.login('abol3nin7744@gmail.com', 'Ahmed12345')

# Set the service URL separately
client.service = 'https://public.api.bsky.app'


# Step 2: Fetch the "What's Hot" feed generator data
data = client.app.bsky.feed.get_feed_generator({
    'feed': 'at://did:plc:z72i7hdynmk6r22z27h6tvur/app.bsky.feed.generator/whats-hot'
})

In [ ]:
# Access feed view information
view = data.view
creator = view.creator
display_name = view.display_name
avatar = view.avatar
print(f"Feed Creator: {creator}")
print(f"Display Name: {display_name}")
print(f"Avatar URL: {avatar}")

Feed Creator: did='did:plc:z72i7hdynmk6r22z27h6tvur' handle='bsky.app' associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='none', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=None, labeler=None, lists=None, starter_packs=None, py_type='app.bsky.actor.defs#profileAssociated') avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:z72i7hdynmk6r22z27h6tvur/bafkreihagr2cmvl2jt4mgx3sppwe2it3fwolkrbtjrhcnwjk4jdijhsoze@jpeg' created_at='2023-04-12T04:53:57.057Z' description='official Bluesky account (check domain👆)\n\nPress: press@blueskyweb.xyz\nSupport: support@bsky.app' display_name='Bluesky' indexed_at='2024-10-17T07:17:00.612Z' labels=[] viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following='at://did:plc:32rgd6jqfnjv4i7527jyq3ng/app.bsky.graph.follow/3l7qugal5432e', known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState') py_type='app.bsky.actor.defs#profileView'

In [ ]:
# Step 3: Define a function to get post details (author, date, text, likes)
def get_post_details(post_rkey, profile_identify=None):
    post_data = client.get_post(post_rkey=post_rkey, profile_identify=profile_identify)
    author = post_data.author
    created_at = post_data.created_at
    text = post_data.text
    likes_count = post_data.likes
    return {
        "author": author,
        "created_at": created_at,
        "text": text,
        "likes": likes_count
    }

In [ ]:
import pandas as pd
from tabulate import tabulate

# Initialize empty lists to store data
text_data = []
created_at_data = []
handle_data = []
like_count_data = []
reply_count_data = []
repost_count_data = []

# Set the desired number of posts
total_posts = 1000

# Fetch posts in batches of 100 until the desired number is reached
cursor = None  # Initialize the cursor for pagination
while len(text_data) < total_posts:
    feed_data = client.get_author_feed(actor='bbcnews-world-rss.bsky.social', limit=100, cursor=cursor)

    for post in feed_data.feed:
        text_data.append(post.post.record.text)
        created_at_data.append(post.post.record.created_at)
        handle_data.append(post.post.author.handle)
        like_count_data.append(post.post.like_count)
        reply_count_data.append(post.post.reply_count)
        repost_count_data.append(post.post.repost_count)

    cursor = feed_data.cursor  # Update the cursor for the next batch
    if cursor is None:  # Break the loop if no more posts are available
        break

# ... (rest of the code to create and save the DataFrame)

In [ ]:
# Print the data in a clean format
for i in range(len(text_data)):
    print(f"Post {i + 1}:")
    print(f"  Text: {text_data[i]}")
    print(f"  Created At: {created_at_data[i]}")
    print(f"  Handle: {handle_data[i]}")
    print(f"  Like Count: {like_count_data[i]}")
    print(f"  Reply Count: {reply_count_data[i]}")
    print(f"  Repost Count: {repost_count_data[i]}")
    print("-" * 20)  # Separator line

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Post 376:
  Text: Hurricane Milton: Man arrested after dog abandoned during storm
  Created At: 2024-10-16T10:02:09.861635Z
  Handle: bbcnews-world-rss.bsky.social
  Like Count: 28
  Reply Count: 1
  Repost Count: 3
--------------------
Post 377:
  Text: Rigathi Gachagua impeachment trial: Case begins to remove Kenya deputy president
  Created At: 2024-10-16T09:07:54.409131Z
  Handle: bbcnews-world-rss.bsky.social
  Like Count: 1
  Reply Count: 0
  Repost Count: 1
--------------------
Post 378:
  Text: Georgia judge blocks ballot hand counting order
  Created At: 2024-10-16T08:49:46.138195Z
  Handle: bbcnews-world-rss.bsky.social
  Like Count: 6
  Reply Count: 0
  Repost Count: 4
--------------------
Post 379:
  Text: 'Only choice I got' - Early voting starts in swing state Georgia
  Created At: 2024-10-16T07:25:39.745762Z
  Handle: bbcnews-world-rss.bsky.social
  Like Count: 5
  Reply Count: 0
  Repost Count: 0
-------

In [ ]:
print(feed_data)

feed=[FeedViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:f2vkoe6mu7hyiyjxwwirkd5l', handle='bbcnews-world-rss.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:f2vkoe6mu7hyiyjxwwirkd5l/bafkreihe2rqbe4ttd5ioxhrxbqnkl34q366qoe2eqejur4xd5ttx4p2gqi@jpeg', created_at='2023-07-23T07:06:23.515Z', display_name='Unofficial BBCNews (World) Bot', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreiawqlxgdggcmuyneldbjjsgeb7jrn5p4n4hovkk3foav66tcd2q5a', indexed_at='2024-09-24T01:13:16.524Z', record=Record(created_at='2024-09-24T01:13:16.524248Z', text='Odisha: India anger over sex abuse allegations in police station', embed=Main(external=External(description='A video of the woman, detailing her alleged abuse by police, has gone

In [ ]:
for post in feed_data.feed:
  print(post.__dict__)

{'post': PostView(author=ProfileViewBasic(did='did:plc:f2vkoe6mu7hyiyjxwwirkd5l', handle='bbcnews-world-rss.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:f2vkoe6mu7hyiyjxwwirkd5l/bafkreihe2rqbe4ttd5ioxhrxbqnkl34q366qoe2eqejur4xd5ttx4p2gqi@jpeg', created_at='2023-07-23T07:06:23.515Z', display_name='Unofficial BBCNews (World) Bot', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreiawqlxgdggcmuyneldbjjsgeb7jrn5p4n4hovkk3foav66tcd2q5a', indexed_at='2024-09-24T01:13:16.524Z', record=Record(created_at='2024-09-24T01:13:16.524248Z', text='Odisha: India anger over sex abuse allegations in police station', embed=Main(external=External(description='A video of the woman, detailing her alleged abuse by police, has gone viral and led 

In [ ]:
for i in range(len(feed_data.feed)):
  # Access the 'text' attribute from the 'post' object
  print(f"(((text))): {feed_data.feed[i].post.record.text}")
  print(f"(((created_at))): {feed_data.feed[i].post.record.created_at}")
  print(f"(((handle))): {feed_data.feed[i].post.author.handle}")
  print(f"(((like_count))): {feed_data.feed[i].post.like_count}")
  print(f"(((reply_count))): {feed_data.feed[i].post.reply_count}")
  print(f"(((repost_count))): {feed_data.feed[i].post.repost_count}")
  print("..................................................................................")

(((text))): Odisha: India anger over sex abuse allegations in police station
(((created_at))): 2024-09-24T01:13:16.524248Z
(((handle))): bbcnews-world-rss.bsky.social
(((like_count))): 4
(((reply_count))): 0
(((repost_count))): 6
..................................................................................
(((text))): Telegram will now provide some user data to authorities
(((created_at))): 2024-09-24T01:12:11.876841Z
(((handle))): bbcnews-world-rss.bsky.social
(((like_count))): 5
(((reply_count))): 0
(((repost_count))): 9
..................................................................................
(((text))): Prince Harry's US visa application will remain private, judge rules
(((created_at))): 2024-09-24T00:59:20.316920Z
(((handle))): bbcnews-world-rss.bsky.social
(((like_count))): 2
(((reply_count))): 0
(((repost_count))): 0
..................................................................................
(((text))): Astronauts return to Earth after longest stay on ISS
((

In [ ]:
from tabulate import tabulate

# Create a list of dictionaries, where each dictionary represents a row in the table
table_data = []
for i in range(len(text_data)):
    table_data.append({
        "Post Number": i + 1,
        "Text": text_data[i],
        "Created At": created_at_data[i],
        "Handle": handle_data[i],
        "Like_count": like_count_data[i],
        "Reply_count": reply_count_data[i],
        "Repost_count": repost_count_data[i]
    })

# Print the table using tabulate
print(tabulate(table_data, headers="keys", tablefmt="grid"))

+---------------+--------------------------------------------------------------------------------------------------------------------------------------+-----------------------------+-------------------------------+--------------+---------------+----------------+
|   Post Number | Text                                                                                                                                 | Created At                  | Handle                        |   Like_count |   Reply_count |   Repost_count |
+===============+======================================================================================================================================+=============================+===============================+==============+===============+================+
|             1 | Who's the famous face behind this E.T. Halloween costume?                                                                            | 2024-11-01T07:54:40.108986Z | bbcnews-world-rss.bsky.socia

In [ ]:
import pandas as pd

# Create a pandas DataFrame
df = pd.DataFrame(table_data)

# Save the DataFrame to a CSV file
df.to_csv('bluesky_data_CSV.csv', index=False)

# Download the CSV file
from google.colab import files
files.download('bluesky_data_CSV.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install openpyxl
import pandas as pd
import openpyxl # Import the openpyxl engine


# Create a pandas DataFrame
df = pd.DataFrame(table_data)

# Save the DataFrame to a CSV file
df.to_excel('bluesky_data_XLSX.xlsx', index=False, engine='openpyxl') # Specify the engine

# Download the CSV file
from google.colab import files
files.download('bluesky_data_XLSX.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>